- 安装此章所需的其他安装包

In [1]:
# pip install -r requirements-extra.txt

# 比较各个 Byte Pair Encoding (BPE)字节对编码 实现

<br>
&nbsp;

## 从 `tiktoken` 使用 BPE  

In [2]:
from importlib.metadata import version

print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.12.0


In [3]:
import tiktoken

tik_tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, world. Is this-- a test?"

In [4]:
integers = tik_tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [5]:
strings = tik_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


In [6]:
print(tik_tokenizer.n_vocab)

50257


<br>
&nbsp;

## 使用chatgpt2最原始的BPE

In [7]:
from bpe_openai_gpt2 import get_encoder, download_vocab

In [8]:
download_vocab()

Fetching encoder.json: 1.04Mit [00:06, 167kit/s]                                                    
Fetching vocab.bpe: 457kit [00:00, 540kit/s]                                                        


In [9]:
orig_tokenizer = get_encoder(model_name="gpt2_model", models_dir=".")

In [12]:
integers = orig_tokenizer.encode(text)

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [13]:
strings = orig_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


<br>
&nbsp;

## 通过Hugging Face变换器使用 BPE

In [14]:
import transformers

transformers.__version__

E:\CODING\LLM_Learning\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'4.57.1'

In [ ]:
from transformers import GPT2Tokenizer

hf_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /gpt2/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000275152B8DC0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 7afb562c-0e22-4583-aed3-b299a36ed910)')' thrown while requesting HEAD https://huggingface.co/gpt2/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /gpt2/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000275152B9150>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 5d12f651-204d-493e-bb78-32ad7e7adfe8)')' thrown while requesting HEAD https://huggingface.co/gpt2/resolve/main/tokenizer_config.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError("HTTPSConnection

In [34]:
hf_tokenizer(strings)["input_ids"]

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]

In [45]:
from transformers import GPT2TokenizerFast

hf_tokenizer_fast = GPT2TokenizerFast.from_pretrained("gpt2")

In [46]:
hf_tokenizer_fast(strings)["input_ids"]

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]

<br>
&nbsp;

## 使用我自己从零开始编写的 BPE 分词器

In [51]:
import os
import sys
import io
import nbformat
import types

def import_from_notebook():
    def import_definitions_from_notebook(fullname, names):
        current_dir = os.getcwd()
        path = os.path.join(current_dir, "..", "05_bpe-from-scratch", fullname + ".ipynb")
        path = os.path.normpath(path)

        # Load the notebook
        if not os.path.exists(path):
            raise FileNotFoundError(f"Notebook file not found at: {path}")

        with io.open(path, "r", encoding="utf-8") as f:
            nb = nbformat.read(f, as_version=4)

        # Create a module to store the imported functions and classes
        mod = types.ModuleType(fullname)
        sys.modules[fullname] = mod

        # Go through the notebook cells and only execute function or class definitions
        for cell in nb.cells:
            if cell.cell_type == "code":
                cell_code = cell.source
                for name in names:
                    # Check for function or class definitions
                    if f"def {name}" in cell_code or f"class {name}" in cell_code:
                        exec(cell_code, mod.__dict__)
        return mod

    fullname = "bpe-from-scratch"
    names = ["BPETokenizerSimple"]

    return import_definitions_from_notebook(fullname, names)

In [52]:
imported_module = import_from_notebook()
BPETokenizerSimple = getattr(imported_module, "BPETokenizerSimple", None)

tokenizer_gpt2 = BPETokenizerSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path=os.path.join("gpt2_model", "encoder.json"),
    bpe_merges_path=os.path.join("gpt2_model", "vocab.bpe")
)

In [53]:
integers = tokenizer_gpt2.encode(text)

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


<br>
&nbsp;

## 快速性能基准测试

In [54]:
with open("../01_main-chapter-code/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

### 原始 OpenAI GPT-2 分词器

In [59]:
%timeit orig_tokenizer.encode(raw_text)

5.56 ms ± 35.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


###  OpenAI GPT-2的Tiktoken 分词器

In [60]:
%timeit tik_tokenizer.encode(raw_text)

1.15 ms ± 9.73 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Hugging Face OpenAI GPT-2 分词器

In [61]:
%timeit hf_tokenizer(raw_text)["input_ids"]

Token indices sequence length is longer than the specified maximum sequence length for this model (5145 > 1024). Running this sequence through the model will result in indexing errors


14.2 ms ± 151 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
%timeit hf_tokenizer(raw_text, max_length=5145, truncation=True)["input_ids"]

14.6 ms ± 139 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
%timeit hf_tokenizer_fast(raw_text)["input_ids"]

Token indices sequence length is longer than the specified maximum sequence length for this model (5145 > 1024). Running this sequence through the model will result in indexing errors


4.84 ms ± 28.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [64]:
%timeit hf_tokenizer_fast(raw_text, max_length=5145, truncation=True)["input_ids"]

4.78 ms ± 53.6 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 自制的 GPT-2 分词器 (用于学习目的)

In [65]:
%timeit tokenizer_gpt2.encode(raw_text)

17.1 ms ± 106 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
